Objective
=========

To train a neural network to segment neurons, we first need data to train it on.
Our approach to this is to simulate imagery from a microscope using the geometry of digitized neuron models.
To do this, we source the models from [NeuroMorpho](https://neuromorpho.org) and load them into the microscope simulator.
This repository comes with a set of SWC files (the neuron models) from NeuroMorpho and can be used for this purpose.

## Setup Logging

In [ ]:
# Configure the logging module with desired format and level
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(name)s | %(levelname)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Create a logger for this notebook
logger = logging.getLogger('segmentation-notebook')
logger.info("Logging configured successfully")

## Open User Configuration

This allows anyone running the notebooks here to customize the operations a bit.

In [ ]:
from userconfig import open_user_config, UserConfig
config: UserConfig = open_user_config()

## Get the File List

In [ ]:
from pathlib import Path
from random import Random
import math
from PIL import Image
import neuroscope

def get_swc_list(path: Path) -> list[Path]:
    entries = []
    for entry in path.glob('*/*.swc'):
        entries.append(entry)
    return entries

swc_paths = get_swc_list(Path(config.swc_dir))
logger.info(f'Found {len(swc_paths)} files.')

## Split File List into Training and Validation Sets

In [ ]:
import random

def train_val_split(paths: list[Path], train_ratio: float = 0.8, seed: int = 0) -> tuple[list[Path], list[Path]]:
    if not 0.0 < train_ratio < 1.0:
        raise ValueError(f"train_ratio must be between 0 and 1; got {train_ratio!r}")
    rng = random.Random(seed)
    shuffled = paths.copy()
    rng.shuffle(shuffled)
    split_idx = int(len(shuffled) * train_ratio)
    train_paths = shuffled[:split_idx]
    val_paths   = shuffled[split_idx:]
    return train_paths, val_paths

train_swc_paths, val_swc_paths = train_val_split(swc_paths)
logger.info(f'Split data into {len(train_swc_paths)} training samples and {len(val_swc_paths)} validation samples.')

## Generate the Training Data

In [ ]:

def open_models(swc_list: list[Path]) -> list[neuroscope.SWCModel]:
    models: list[neuroscope.SWCModel] = []
    for swc_path in swc_list:
        model = neuroscope.SWCModel()
        if not model.load_from_file(str(swc_path)):
            logger.error(f'failed to open {swc_path}, skipping.')
            pass
        models.append(model)
    return models

from tqdm.notebook import tqdm
from itertools import product

def render_models(swc_models: list[neuroscope.SWCModel], out_dir: Path, res: tuple[int, int] = (512, 512), fov=600, seed: int = 0):
    out_dir.mkdir(exist_ok=True)
    seg_scope = neuroscope.SegmentationMicroscope(res[0], res[1], vertical_fov=fov)
    scope = neuroscope.FluorescenceMicroscope(res[0], res[1], vertical_fov=fov)
    counter = 0
    num_augmentations = 40
    rng = Random(seed)
    num_models = len(swc_models)
    num_iterations = num_models * num_augmentations
    loop = tqdm(product(range(num_models), range(num_augmentations)), total=num_iterations)
    for model_index, augmentation_index in loop:
        model = swc_models[model_index]

        tissue_config = neuroscope.TissueConfig()
        tissue_config.coverage = rng.uniform(0.6, 1.0)
        tissue_config.max_density = rng.uniform(0.01, 0.08)
        tissue_config.seed = rng.randint(0, 1000)

        tissue = neuroscope.Tissue()
        tissue.set_config(tissue_config)

        transform = neuroscope.Transform()
        transform.position = neuroscope.Vec3f(rng.uniform(-100, 100), rng.uniform(-100, 100), 0)
        transform.rotation = neuroscope.Vec3f(0, 0, rng.uniform(-math.pi, math.pi))

        # segmentation
        seg_scope.capture(model, tissue, transform)
        seg_buffer = seg_scope.copy_rgb_buffer()
        seg_path = out_dir / f'{counter:05}_mask.png'
        Image.frombytes('RGB', res, seg_buffer).save(seg_path)

        # real
        scope.capture(model, tissue, transform)
        real_buffer = scope.copy_buffer()
        real_path = out_dir / f'{counter:05}.png'
        Image.frombytes('L', res, real_buffer).save(real_path)
        loop.set_description(f'Model: {model_index}/{num_models}, Augmentation: {augmentation_index}/{num_augmentations}')

        counter += 1

out_dir = Path(config.imagery_dir)
out_dir.mkdir(exist_ok=True)
logger.info('Starting rendering process. This may take a while. A good time to nap at your desk.')
render_models(open_models(train_swc_paths), out_dir / 'train')
logger.info('Training samples done rendering.')
render_models(open_models(val_swc_paths), out_dir / 'val')
logger.info('Validation samples done rendering.')